In [ ]:
#Open using https://colab.research.google.com/github/MaartenSmeets/insightcode/blob/develop/workshop.ipynb
!git clone https://github.com/MaartenSmeets/insightcode.git

In [ ]:
#Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
#Start server
get_ipython().system_raw('ollama serve &')

In [ ]:
import subprocess

def pull_model_silently(model_name):
    """Pull a model using ollama and suppress output."""
    try:
        # Run the ollama pull command, suppressing the output
        result = subprocess.run(
            ["ollama", "pull", model_name], 
            stdout=subprocess.DEVNULL,  # Suppress standard output
            stderr=subprocess.DEVNULL   # Suppress error output
        )
        
        # Check if the command was successful
        if result.returncode == 0:
            print(f"Model '{model_name}' downloaded successfully.")
        else:
            print(f"Failed to download model '{model_name}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
pull_model_silently("deepseek-coder-v2:16b-lite-instruct-q5_K_M")